In [1]:
%load_ext autoreload
%autoreload 2
import matplotlib.pyplot as plt
import numpy as np
from copy import deepcopy
import pandas as pd
import time, sys, os

# short decision tree

In [ ]:
from sklearn.tree import DecisionTreeRegressor, plot_tree

# load in same data
boston_data = pd.read_csv("../data/boston.csv", index_col=0)
y = boston_data.medv.values
X = boston_data.drop("medv", axis=1)
features = X.columns
X = X.values

# specify a decision tree with a maximum depth
dt = DecisionTreeRegressor(max_depth=3)
dt.fit(X, y)

# calculat mse on the training data
preds = dt.predict(X)
print(f'train mse: {np.mean(np.square(preds-y)):0.2f}')

plot_tree(dt)
plt.savefig('tree.pdf')
plt.show()

# integer linear models

In [7]:
sys.path.append('../imodels/slim')
from SLIM import SLIM

In [8]:
np.random.seed(123)

# generate X and y
n, p = 1000, 10
X = np.random.randn(n, p)
y = X[:, 0] + 2 * X[:, 1] - 1 * X[:, 2] + np.random.randn(n)

# fit linear models with different regularization parameters
model = SLIM()
for lambda_reg in [0, 1e-2, 5e-2, 1e-1, 1, 2]:
    model.fit(X, y, lambda_reg)
    mse = np.mean(np.square(y - model.predict(X)))
    print(f'lambda: {lambda_reg}\tmse: {mse: 0.2f}\tweights: {model.coef_}')

lambda: 0	mse:  2.09	weights: [ 1  1 -1  0  0  0  0  0  0  0]
lambda: 0.01	mse:  2.09	weights: [ 1  1 -1  0  0  0  0  0  0  0]
lambda: 0.05	mse:  1.02	weights: [ 1  2 -1  0  0  0  0  0  0  0]
lambda: 0.1	mse:  1.02	weights: [ 1  2 -1  0  0  0  0  0  0  0]
lambda: 1	mse:  3.08	weights: [ 0  1 -1  0  0  0  0  0  0  0]
lambda: 2	mse:  1.90	weights: [ 0  2 -1  0  0  0  0  0  0  0]


# rulefit
- installed with: `pip install git+git://github.com/christophM/rulefit.git`
- [documentation](https://github.com/christophM/rulefit) and the [original paper](http://statweb.stanford.edu/~jhf/ftp/RuleFit.pdf)

In [9]:
from rulefit import RuleFit

# load some data
boston_data = pd.read_csv("../data/boston.csv", index_col=0)
y = boston_data.medv.values
X = boston_data.drop("medv", axis=1)
features = X.columns
X = X.values

# fit a rulefit model
rf = RuleFit()
rf.fit(X, y, feature_names=features)

# calculate mse on the training data
preds = rf.predict(X)
print(f'train mse: {np.mean(np.square(preds-y)):0.2f}')

train mse: 1.45


/accounts/projects/vision/.local/lib/python3.7/site-packages/sklearn/linear_model/coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 79.97186577065486, tolerance: 4.2716295415019765
  positive)


now, let's inspect the rules

In [10]:
rules = rf.get_rules()

rules = rules[rules.coef != 0].sort_values("support", ascending=False)

# 'rule' is how the feature is constructed
# 'coef' is its weight in the final linear model
# 'support' is how many points it applies to
rules[['rule', 'coef', 'support']].head().style.background_gradient(cmap='viridis')

,rule,coef,support
1,zn,0.00720871,1
8,rad,0.0402382,1
6,age,-0.0377822,1
1217,rm <= 8.752500057220459 & indus <= 26.69499969482422,0.90752,0.987179
1193,dis > 1.1716500520706177 & rm <= 8.168499946594238 & rm <= 8.752500057220459,-1.88072,0.978632


# scalable bayesian rule lists
- docs are [here](https://github.com/csinva/interpretability-workshop/tree/master/models/bayesian_rule_lists)

In [13]:
sys.path.append('../imodels/bayesian_rule_lists')
sys.path.append('../imodels/bayesian_rule_lists/discretization')
from RuleListClassifier import *
from sklearn.datasets import fetch_openml
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


feature_labels = ["#Pregnant","Glucose concentration test","Blood pressure(mmHg)","Triceps skin fold thickness(mm)",
                  "2-Hour serum insulin (mu U/ml)","Body mass index","Diabetes pedigree function","Age (years)"]

In [14]:
np.random.seed(13)
data = fetch_openml("diabetes") # get dataset
y = (data.target == 'tested_positive').astype(np.int) # labels 0-1

Xtrain, Xtest, ytrain, ytest = train_test_split(data.data, y, test_size=0.75) # split

# train classifier (allow more iterations for better accuracy; use BigDataRuleListClassifier for large datasets)
print('training...')
model = RuleListClassifier(max_iter=10000, class1label="diabetes", verbose=False)
model.fit(Xtrain, ytrain, feature_labels=feature_labels)

print("RuleListClassifier Accuracy:", model.score(Xtest, ytest), "Learned interpretable model:\n", model)
# print("RandomForestClassifier Accuracy:", RandomForestClassifier(n_estimators=10).fit(Xtrain, ytrain).score(Xtest, ytest))

training...
RuleListClassifier Accuracy: 0.7065972222222222 Learned interpretable model:
 Trained RuleListClassifier for detecting diabetes
IF #Pregnant : 6.5_to_inf THEN probability of diabetes: 65.7% (49.5%-80.3%)
ELSE IF Glucose concentration test : -inf_to_122.5 THEN probability of diabetes: 9.9% (4.9%-16.4%)
ELSE IF Body mass index : -inf_to_30.9 THEN probability of diabetes: 36.0% (18.8%-55.3%)
ELSE probability of diabetes: 69.2% (54.1%-82.5%)



# optimal classification tree
- docs [here](https://github.com/csinva/interpretability-workshop/tree/master/models/optimal_classification_tree)
- note: this implementation is still somewhat unstable, and can be made faster by installing either `cplex` or `gurobi`

In [3]:
sys.path.append('../imodels/optimal_classification_tree/pyoptree')
sys.path.append('../imodels/optimal_classification_tree/')

In [ ]:
from optree import OptimalTreeModel
feature_names = np.array(["x1", "x2"])

X = np.array([[1, 2, 2, 2, 3], [1, 2, 1, 0, 1]]).T
y = np.array([1, 1, 0, 0, 0]).reshape(-1, 1)
X_test = np.array([[1, 1, 2, 2, 2, 3, 3], [1, 2, 2, 1, 0, 1, 0]]).T
y_test = np.array([1, 1, 1, 0, 0, 0, 0])

np.random.seed(13)
model = OptimalTreeModel(tree_depth=3, N_min=1, alpha=0.1) #, solver_name='baron'
model.fit(X_test, y_test) # this method is currently using the fast, but not optimal solver
preds = model.predict(X_test)

# fit on the bigger diabetes dset from above
# model.fit(Xtrain, ytrain) # this method is currently using the fast, but not optimal solver
# preds = model.predict(Xtest)

print('acc', np.mean(preds == y_test))

In [10]:
model.print_tree(feature_names)

depth 0:
	 x2 > 0.8114524236945737

depth 1:
	node 2 undefined
	node 3 undefined

depth 2:
	node 4 undefined
	 x1 > 0.01086684288089712
	 x2 > 0.9159532769401844
	node 7 undefined

depth 3:
	node 8 undefined
	node 9 undefined
	node 10 undefined
	node 11 undefined
	node 12 undefined
	node 13 undefined
	node 14 undefined
	node 15 undefined

